In [2]:
# Import libraries
from __future__ import print_function


import numpy as np
import pandas as pd
import psycopg2
import socket
import sys
import os
import getpass

from collections import OrderedDict

from sepsis_utils import sepsis_utils as su
from sepsis_utils import roc_utils as ru

# path to output data files to - can be relative to the current folder
data_path = 'data'

In [3]:
# colours for prettier plots
import matplotlib
import matplotlib.pyplot as plt
import colorsys
def gg_color_hue(n):
    hues = np.linspace(15, 375, n)
    hsv_tuples = [(x*1.0/360.0, 0.5, 0.8) for x in hues]
    rgb_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples)
    return rgb_tuples

marker = ['v','o','d','^','s','o','+']
ls = ['-','-','-','-','-','s','--','--']

# plot settings
%matplotlib inline
plt.style.use('ggplot')
font = {'size'   : 20}
matplotlib.rc('font', **font)

In [4]:
# helper function for generating tables
# this rolls back the cursor if it fails
def execute_query_safely(sql, con):
    cur = con.cursor()
    
    # try to execute the query
    try:
        cur.execute(sql)
    except:
        # if an exception, rollback, rethrow the exception - finally closes the connection
        cur.execute('rollback;')
        raise
    finally:
        cur.close()
    
    return

In [5]:
sqluser = getpass.getuser()
print('Using username {}'.format(sqluser))

dbname = 'mimic'
schema_name = 'mimiciii'
query_schema = 'SET search_path to public,' + schema_name + ';'

Using username work


In [7]:
# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user='postgres', password="19910422")

print('Connected to postgres {}.{}.{}!'.format(int(con.server_version/10000),
                                              (con.server_version - int(con.server_version/10000)*10000)/100,
                                              (con.server_version - int(con.server_version/100)*100)))

Connected to postgres 15.0.02.2!


In [28]:
# check if the sepsis3_cohort table exists ... if not we must generate it
query = """
SELECT EXISTS(SELECT 1 FROM information_schema.tables 
              WHERE table_catalog = '{}'
              AND table_schema in ('public','{}')
              AND table_name = 'sepsis3');
""".format(dbname, schema_name)
tbl_exists = pd.read_sql_query(query, con)
tbl_exists = tbl_exists.loc[0,'exists']
if tbl_exists:
    print('Found the `sepsis3` table. Skipping generation of data in SQL.')
else:
    print('Running SQL code to generate tables. This may take some time.')
    
    # read through the "make-tables.sql" file in the sql subfolder
    query_path = 'query'
    
    with open(os.path.join(query_path, 'make-tables.sql'), 'r') as fp:
        for line in fp.readlines():
            if len(line)<2:
                print(line,end='')
                continue
            
            if line[0:2] != '\i':
                print(line,end='')
                continue
                
            # lines which begin with '\i' call SQL files that generate tables
            query_file = os.path.join(query_path, line[3:].replace('\n',''))
            print('Running {} ...'.format(query_file), end=' ')
            with open(query_file, 'r') as fp_query:
                query = ''.join(fp_query.readlines())
            execute_query_safely(query_schema + query, con)
            print('done.')

/var/folders/__/gxsm7_5x7215zjff14wnpntm0000gn/T/ipykernel_22758/3381836025.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tbl_exists = pd.read_sql_query(query, con)


Running SQL code to generate tables. This may take some time.
-- As the script is generating many tables, it may take some time.

-- We assume the database and the search path are set correctly.
-- You can set the search path as follows:
-- SET SEARCH_PATH TO public,mimiciii;
-- This will create tables on public and read tables from mimiciii

BEGIN;
-- ----------------------------- --
-- ---------- STAGE 1 ---------- --
-- ----------------------------- --

Running query/tbls/abx-poe-list.sql ... done.
Running query/tbls/abx-micro-prescription.sql ... done.
Running query/tbls/suspicion-of-infection.sql ... done.

-- blood cultures around ICU admission
Running query/tbls/blood-culture-icu-admit.sql ... done.

-- generate cohort
Running query/tbls/cohort.sql ... done.

-- generate sepsis definitions
Running query/../mimic-code/concepts/sepsis/angus.sql ... done.
Running query/../mimic-code/concepts/sepsis/martin.sql ... done.
Running query/../mimic-code/concepts/sepsis/explicit.sql ... do

In [29]:
# exclusion criteria:
#   - less than 16 years old
#   - never have any chartevents data (i.e. likely administrative error)
#   - not cardiac surgery
#   - suspected of infection
#   - first ICU stay
#   - not a CareVue patient (i.e. admitted 2008-2012)
# these exclusion criteria are created in the sepsis3_cohort table
query = query_schema + "select * from sepsis3_cohort"
co = pd.read_sql_query(query,con)

/var/folders/__/gxsm7_5x7215zjff14wnpntm0000gn/T/ipykernel_22758/324908962.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  co = pd.read_sql_query(query,con)


In [30]:
co

,hadm_id,icustay_id,intime,outtime,age,gender,ethnicity,first_service,dbsource,suspected_of_infection_poe,...,positiveculture_poe,antibiotic_time_poe,exclusion_secondarystay,exclusion_nonadult,exclusion_csurg,exclusion_carevue,exclusion_early_suspicion,exclusion_late_suspicion,exclusion_bad_data,excluded
0,152234,200001,2181-11-25 19:06:12,2181-11-28 20:59:25,61.0499,F,ASIAN - ASIAN INDIAN,MED,metavision,1,...,0.0,2181-11-18,1,0,0,0,1,0,0,1
1,163557,200003,2199-08-02 19:50:04,2199-08-08 17:09:18,48.2940,M,WHITE,SURG,carevue,1,...,1.0,2199-08-02,0,0,0,1,0,0,0,1
2,189514,200006,2159-09-03 11:28:14,2159-09-04 19:08:10,54.0710,M,OTHER,MED,carevue,0,...,NaN,NaT,1,0,0,1,0,0,0,1
3,129310,200007,2109-02-17 10:03:37,2109-02-18 17:03:12,43.3439,M,WHITE,CMED,carevue,0,...,NaN,NaT,0,0,0,1,0,0,0,1
4,129607,200009,2189-11-30 10:34:32,2189-12-02 14:17:37,47.5548,F,WHITE,CSURG,carevue,1,...,NaN,2189-11-30,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,169431,299993,2149-11-13 21:01:05,2149-11-16 16:57:30,26.2264,M,WHITE,SURG,carevue,0,...,NaN,NaT,0,0,0,1,0,0,0,1
61528,177406,299994,2114-12-09 11:56:00,2114-12-15 20:29:14,74.2165,F,WHITE,MED,carevue,1,...,0.0,2114-12-13,1,0,0,1,0,1,0,1
61529,134959,299995,2116-03-04 17:44:39,2116-03-06 18:03:55,23.3763,M,WHITE,NMED,carevue,1,...,0.0,2116-03-04,0,0,0,1,0,0,0,1
61530,158288,299998,2181-07-05 18:47:40,2181-07-07 17:24:38,86.9560,M,WHITE,GU,metavision,1,...,0.0,2181-07-05,0,0,0,0,0,0,0,0


# Exclusions - applied independently

In [32]:
# print out the exclusions
print('Cohort - initial size: {} ICU stays'.format(co.shape[0]))
idxRem = np.zeros(co.shape[0])
for c in co.columns:
    if c.startswith('exclusion_'):
        print('  {:5g} ({:2.2f}%) - {}'.format(np.sum(co[c]),np.mean(co[c])*100.0, c))
        idxRem[co[c].values==1] = 1
        
print('Final cohort size: {} ICU stays ({:2.2f}%).'.format(co.shape[0] - np.sum(idxRem), (1-np.mean(idxRem))*100.0))

Cohort - initial size: 61532 ICU stays
  15056 (24.47%) - exclusion_secondarystay
   8109 (13.18%) - exclusion_nonadult
   7343 (11.93%) - exclusion_csurg
  37912 (61.61%) - exclusion_carevue
   9550 (15.52%) - exclusion_early_suspicion
   2808 (4.56%) - exclusion_late_suspicion
    491 (0.80%) - exclusion_bad_data
Final cohort size: 11325.0 ICU stays (18.41%).


# Exclusions - applied sequentially

In [33]:
# print out the exclusions *SEQUENTIALLY* - i.e. if already excluded, don't re-print
print('Cohort - initial size: {} ICU stays'.format(co.shape[0]))

COL_REM = ['exclusion_nonadult', 'exclusion_secondarystay',
           'exclusion_csurg','exclusion_carevue',
           'exclusion_early_suspicion', 'exclusion_late_suspicion', 'exclusion_bad_data']

idxRem = np.zeros(co.shape[0])
for c in COL_REM:
    N_REM = np.sum( (idxRem == 0) & (co[c].values==1) )
    print('  {:5g} ({:2.2f}%) - {}'.format(N_REM,N_REM*100.0/co.shape[0], c))
    idxRem[co[c].values==1] = 1
        
print('Final non-suspected cohort size: {:g} ICU stays ({:2.2f}%).'.format(co.shape[0] - np.sum(idxRem), (1-np.mean(idxRem))*100.0))

print()
# define idxRem to exclude non-metavision admissions initially
print('=============================')
print('====== METAVISION ONLY ======')
print(' Patients admitted 2008-2012 ')
print('=============================')
print()

idxRem = co['dbsource']!='metavision'
N = np.sum(~idxRem)
N_IGNORE = np.sum(idxRem)

# print out the exclusions *SEQUENTIALLY* - i.e. if already excluded, don't re-print
print('Cohort - initial size: {} ICU stays'.format(N))

for c in COL_REM:
    N_REM = np.sum( (idxRem == 0) & (co[c].values==1) )
    print('  {:5g} ({:2.2f}%) - {}'.format(N_REM,N_REM*100.0/N, c))
    idxRem[co[c].values==1] = True
        
print('Final non-suspected cohort size: {}/{} ICU stays ({:2.2f}%).'.format(
        np.sum(~idxRem), N, np.sum(~idxRem)*100.0/N))
print('')

print('Of these patients...')
c='suspected_of_infection_poe'
N_REM = np.sum( (~idxRem) & (co[c].values==1) )
print('  {:5g} ({:2.2f}%) - {}'.format(N_REM, N_REM*100.0/ np.sum(~idxRem), c))

Cohort - initial size: 61532 ICU stays
   8109 (13.18%) - exclusion_nonadult
  14826 (24.09%) - exclusion_secondarystay
   5905 (9.60%) - exclusion_csurg
  18909 (30.73%) - exclusion_carevue
   1719 (2.79%) - exclusion_early_suspicion
    721 (1.17%) - exclusion_late_suspicion
     18 (0.03%) - exclusion_bad_data
Final non-suspected cohort size: 11325 ICU stays (18.41%).

====== METAVISION ONLY ======
 Patients admitted 2008-2012 

Cohort - initial size: 23620 ICU stays
      3 (0.01%) - exclusion_nonadult
   7536 (31.91%) - exclusion_secondarystay
   2298 (9.73%) - exclusion_csurg
      0 (0.00%) - exclusion_carevue
   1719 (7.28%) - exclusion_early_suspicion
    721 (3.05%) - exclusion_late_suspicion
     18 (0.08%) - exclusion_bad_data
Final non-suspected cohort size: 11325/23620 ICU stays (47.95%).

Of these patients...
   7122 (62.89%) - suspected_of_infection_poe


# Histogram of time of suspected infection

In [35]:
# # generate exclusions *except* early/late suspicion
# COL_REM = ['exclusion_nonadult', 'exclusion_secondarystay',
#            'exclusion_csurg','exclusion_carevue',
#            #'exclusion_early_suspicion', 'exclusion_late_suspicion',
#            'exclusion_bad_data']
# idxRem = np.zeros(co.shape[0])
# for c in COL_REM:
#     N_REM = np.sum( (idxRem == 0) & (co[c].values==1) )
#     idxRem[co[c].values==1] = 1
    
# idxRem = idxRem.astype(bool)

# # distribution of time of infection
# N_HR = 96
# xi = np.linspace(-N_HR, N_HR, N_HR*2+1)
# col = gg_color_hue(5)

# pretty_dict = {'suspected_infection_time_poe_days': 'Blood culture + ABX'}
# c = 'suspected_infection_time_poe_days'

# idxKeep = ~co[c].isnull()
# tmp = co.loc[~idxRem & idxKeep, c].values * 24.0
# N_firstday = sum( (tmp>-24) & (tmp<24) )

# lbl_str = pretty_dict[c]
# lbl_str += '\n' + '{:2.2f}% of grp >  24hr.'.format(sum(tmp>24)*100.0 / np.sum(~idxRem & idxKeep))
# lbl_str += '\n' + '{:2.2f}% of grp  +-24hr.'.format(sum((tmp>=-24) & (tmp<=24))*100.0 / np.sum(~idxRem & idxKeep))
# lbl_str += '\n' + '{:2.2f}% of grp < -24hr.'.format(sum(tmp<-24)*100.0 / np.sum(~idxRem & idxKeep))

# plt.figure(figsize=[14,6])
# plt.hist( tmp, bins=xi, label=lbl_str, normed=True )
# plt.title('{} ({:2.2f}%) in [-24,24]. '.format(
#         N_firstday, N_firstday*100.0 / np.sum(~idxRem)))
# plt.legend(loc='upper left')
    
# plt.show()

We can see that most patients are suspected of infection either before, or at the time of their ICU admission. This motivates the decision to evaluate the performance of the scores at ICU admission.

# Load final dataset from Postgres

In [36]:
# load in final dataset - note we apply the exclusion criteria with excluded=0
query = query_schema + "select * from sepsis3 where excluded = 0"

df = pd.read_sql_query(query,con)

/var/folders/__/gxsm7_5x7215zjff14wnpntm0000gn/T/ipykernel_22758/2017747276.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,con)


In [37]:
df

,icustay_id,hadm_id,excluded,intime,outtime,dbsource,suspected_infection_time_poe,suspected_infection_time_poe_days,specimen_poe,positiveculture_poe,...,sepsis_cdc_simple,elixhauser_hospital,vent,sofa,lods,sirs,qsofa,qsofa_sysbp_score,qsofa_gcs_score,qsofa_resprate_score
0,200021,109307,0,2114-12-26 19:45:12,2114-12-27 22:46:28,metavision,NaT,NaN,None,NaN,...,0,0,1,4,7,3,3,1.0,1.0,1.0
1,200028,181955,0,2133-10-29 17:13:50,2133-11-01 14:55:14,metavision,NaT,NaN,None,NaN,...,0,0,1,10,6,3,2,1.0,0.0,1.0
2,200033,198650,0,2198-08-07 17:56:17,2198-08-21 14:59:18,metavision,2198-08-08 01:00:00,-0.294248,MRSA SCREEN,0.0,...,1,9,0,1,1,2,0,0.0,0.0,0.0
3,200061,121149,0,2134-01-23 16:38:46,2134-01-25 16:59:14,metavision,2134-01-24 00:00:00,-0.306412,MRSA SCREEN,0.0,...,0,0,0,2,2,4,1,0.0,0.0,1.0
4,200075,132255,0,2159-09-23 00:13:20,2159-09-25 01:55:17,metavision,2159-09-23 00:00:00,0.009259,MRSA SCREEN,0.0,...,1,0,1,8,4,3,2,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11320,299948,183226,0,2119-05-25 13:47:31,2119-05-27 12:31:54,metavision,2119-05-25 14:37:00,-0.034363,MRSA SCREEN,0.0,...,1,0,0,3,2,2,2,1.0,0.0,1.0
11321,299949,199962,0,2118-07-16 05:42:45,2118-07-20 14:46:25,metavision,2118-07-16 00:00:00,0.238021,SWAB,0.0,...,0,0,1,3,2,2,0,0.0,0.0,0.0
11322,299950,110990,0,2122-06-20 13:25:29,2122-06-22 18:21:02,metavision,2122-06-20 00:00:00,0.559363,BLOOD CULTURE,0.0,...,1,0,0,5,2,2,2,1.0,0.0,1.0
11323,299962,146497,0,2195-11-25 19:04:06,2195-11-26 13:21:12,metavision,NaT,NaN,None,NaN,...,0,0,0,5,6,2,2,1.0,0.0,1.0


We have: ICU intime/outtime, suspected infection time, whether the microbiology culture was positive, some demographics, comorbidities, outcomes, and the severity scores. 

The severity scores are extracted at a [0, 24] hour window centered around ICU admission - except labs have an extended [-6, 24] hour window (i.e. 'sofa' is extracted in this way).

## Add in useful variables

In [13]:
# add the composite outcome
df['composite_outcome'] = ( (df['hospital_expire_flag']==1) | (df['icu_los']>=3) ).astype(int)

labels = OrderedDict([['suspicion_poe', 'BC + ABX (Prescribed)']])

# add some other useful variables
df['blood culture'] = (~df['blood_culture_time'].isnull())
df['suspicion_poe'] = (~df['suspected_infection_time_poe_days'].isnull())

df['abx_poe'] = (~df['antibiotic_time_poe'].isnull())

df['sepsis-3'] = ((df['suspicion_poe']==1) & (df['sofa']>=2)).astype(int)
df['sofa>=2'] = (df['sofa']>=2).astype(int)

# Baseline model + scores

The original paper evaluates a *baseline model* with the addition of the various severity scores. 

> To measure predictive validity, a baseline risk model was created for in-hospital mortality based on preinfection criteria using multivariable logistic regression. The baseline model included age (as a fractional polynomial), sex, race/ethnicity (black, white, or other), and the weighted Charlson comorbidity score (as fractional polynomial) as a measure of chronic comorbidities.

This baseline model includes:

* age (fractional polynomial)
* sex
* ethnicity
* Charlson comorbidities (fractional polynomial)

We can reproduce this model (done in the appendix), with the following caveats:

1. We will build and evaluate the model on the same dataset, so our estimates are "apparent"
2. We will use Elixhauser comorbidities, not Charlson comorbidities
3. We may not have identical fractional polynomial terms (as we are rebuilding the model on our dataset)

The following code block extracts the covariates for the baseline model. R code in the repository can use this file to build a fractional polynomial model.

In [14]:
X_header = ['age','elixhauser_hospital','hospital_expire_flag','sepsis_angus',
            'is_male','race_black','race_other',
            'qsofa','sofa','sirs','lods']

X = df[X_header].values

# we'll write out the design matrix for the MFP model here - this is used by the R code
np.savetxt(os.path.join(data_path, 'sepsis3-design-matrix.csv'), X, fmt='%4.4f',
           delimiter=',', header=','.join(X_header), comments='')

# Save the data to file

The dataframes will be loaded directly from a file, rather than the database.

The `X` data which was written to 'sepsis3-design-matrix.csv' will be used by the `print_auc_table_baseline` function to evaluate the AUROC of the scores when incorporated with the baseline model.

In [15]:
df.to_csv(os.path.join(data_path, 'sepsis3-df.csv'),sep=',',index=False)

# Dataset with no exclusions

It may be useful for others to analyze the dataset without exclusions. Here we generate an identical copy of the data, except for all `icustay_id` in MIMIC-III.

In [38]:
# load in final dataset - note we add in the individual exclusion criteria
query = query_schema + """
select ie.subject_id
, s.*
, co.exclusion_secondarystay
, co.exclusion_nonadult
, co.exclusion_csurg
, co.exclusion_carevue
, co.exclusion_early_suspicion
, co.exclusion_late_suspicion
, co.exclusion_bad_data
from sepsis3 s
-- add in subject_id
inner join icustays ie
  on s.icustay_id = ie.icustay_id
inner join sepsis3_cohort co
  on s.icustay_id = co.icustay_id
order by s.icustay_id
"""

df = pd.read_sql_query(query,con)

# add the composite outcome
df['composite_outcome'] = ( (df['hospital_expire_flag']==1) | (df['icu_los']>=3) ).astype(int)

labels = OrderedDict([['suspicion_poe', 'BC + ABX (Prescribed)']])

# add some other useful variables
df['blood culture'] = (~df['blood_culture_time'].isnull())
df['suspicion_poe'] = (~df['suspected_infection_time_poe_days'].isnull())

df['abx_poe'] = (~df['antibiotic_time_poe'].isnull())

df['sepsis-3'] = ((df['suspicion_poe']==1) & (df['sofa']>=2)).astype(int)
df['sofa>=2'] = (df['sofa']>=2).astype(int)

# df.to_csv(os.path.join(data_path, 'sepsis3-df-no-exclusions.csv'),sep=',',index=False)

/var/folders/__/gxsm7_5x7215zjff14wnpntm0000gn/T/ipykernel_22758/311305226.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,con)


In [46]:
df[(df['sepsis-3'] == 1)]['icustay_id'].nunique()

34070

In [41]:
df.columns

Index(['subject_id', 'icustay_id', 'hadm_id', 'excluded', 'intime', 'outtime',
       'dbsource', 'suspected_infection_time_poe',
       'suspected_infection_time_poe_days', 'specimen_poe',
       'positiveculture_poe', 'antibiotic_time_poe', 'blood_culture_time',
       'blood_culture_positive', 'age', 'gender', 'is_male', 'ethnicity',
       'race_white', 'race_black', 'race_hispanic', 'race_other',
       'metastatic_cancer', 'diabetes', 'height', 'weight', 'bmi',
       'first_service', 'hospital_expire_flag', 'thirtyday_expire_flag',
       'icu_los', 'hosp_los', 'sepsis_angus', 'sepsis_martin',
       'sepsis_explicit', 'septic_shock_explicit', 'severe_sepsis_explicit',
       'sepsis_nqf', 'sepsis_cdc', 'sepsis_cdc_simple', 'elixhauser_hospital',
       'vent', 'sofa', 'lods', 'sirs', 'qsofa', 'qsofa_sysbp_score',
       'qsofa_gcs_score', 'qsofa_resprate_score', 'exclusion_secondarystay',
       'exclusion_nonadult', 'exclusion_csurg', 'exclusion_carevue',
       'exclusion_ear

In [17]:
con.close()